In [8]:
# Author: Sunny You
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf 
from tensorflow import keras
import shap

In [9]:
os.chdir('/Users/sunnyyou/Documents/Real_Time_HAI/HIPCNN/IMERG')

In [181]:
train = pd.read_csv("DEV/P06_2018_train_resample.csv")
train = train[["GIS_ID", "DATE", "SHIPS_PER", "SHIPS_POT", "VMAX", "SHDC", "IC", "Category"]]
# train = train.drop(["VMAX_FT"], axis = 1)
train.columns = ["GIS_ID", "DATE", "PER", "POT", "VMAX", "SHDC_FT", "IC", "Category"]
train

,GIS_ID,DATE,PER,POT,VMAX,SHDC_FT,IC,Category
0,ATL_200002_TD_2000062318,2000062318,9.310000e-09,84.0,30,25.300001,0,TD
1,ATL_200002_TD_2000062400,2000062400,0.000000e+00,84.0,30,18.400000,0,TD
2,ATL_200002_TD_2000062406,2000062406,0.000000e+00,84.0,30,16.100000,0,TD
3,ATL_200002_TD_2000062412,2000062412,0.000000e+00,85.0,30,10.100000,0,TD
4,ATL_200002_TD_2000062418,2000062418,0.000000e+00,87.0,30,4.300000,0,TD
...,...,...,...,...,...,...,...,...
9155,ATL_201011_C4_2010091406,2010091406,-1.000000e+01,15.0,115,6.500000,0,Maj
9156,ATL_200608_C3_2006091500,2006091500,-5.000000e+00,12.0,100,18.700001,-5,Maj
9157,ATL_200013_C3_2000092400,2000092400,3.500000e+01,19.0,105,3.300000,-5,Maj
9158,ATL_201905_C3_2019090506,2019090506,1.000000e+01,30.0,100,17.700001,0,Maj


In [182]:
test = pd.read_csv("DEV/P06_2018_test.csv")
test = test[["GIS_ID", "DATE", "SHIPS_PER", "SHIPS_POT", "VMAX", "SHDC", "IC", "Category"]]
# test = test.drop(["VMAX_FT"], axis = 1)
test.columns = ["GIS_ID", "DATE", "PER", "POT", "VMAX", "SHDC_FT", "IC", "Category"]
test

,GIS_ID,DATE,PER,POT,VMAX,SHDC_FT,IC,Category
0,ATL_201801_TS_2018052800,2018052800,15.0,67.0,55,8.700000,0,TS
1,ATL_201801_TS_2018052806,2018052806,10.0,63.0,55,11.000000,-5,TS
2,ATL_201801_TS_2018052812,2018052812,-5.0,66.0,50,20.000000,-5,TS
3,ATL_201802_TS_2018070506,2018070506,5.0,52.0,35,7.300000,5,TS
4,ATL_201802_TS_2018070512,2018070512,5.0,51.0,40,6.100000,5,TS
...,...,...,...,...,...,...,...,...
425,ATL_201920_TS_2019112218,2019112218,5.0,38.0,55,40.299999,5,TS
426,ATL_201920_TS_2019112300,2019112300,10.0,24.0,60,41.400002,0,TS
427,ATL_201920_TS_2019112306,2019112306,5.0,21.0,60,42.600002,-5,TS
428,ATL_201920_TS_2019112312,2019112312,-5.0,12.0,55,33.500000,0,TS


In [183]:
# # train = pd.concat([train, test])
# test = pd.DataFrame(np.reshape(['ATL_202313_C3_2023090718', '2023090718', 35, 54, 105, 9.9, -999, 'C3'], (1, 8)))
# test.columns = ('GIS_ID', 'DATE', 'PER', 'POT', 'VMAX', 'SHDC_FT', 'IC', 'Category')
# train = train.iloc[:, :-1]
# train = train.reset_index(drop = False)
# print(train.head(), train.shape)
# print(test.head(), test.shape)

In [184]:
train_img = []
train_ships = []
train_label = []
test_img = []
test_ships = []
test_label = []
for f in range(len(train.GIS_ID)) :
    filename = "IMERG_CSV/" + train.GIS_ID[f] + ".csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[40:81]
        temp = temp.iloc[:, 40:81]
        temp = np.array(temp)
        train_img.append(temp)
        lab = train.IC[f] + train.VMAX[f]
        train_label.append(lab)
        ships = np.array([train.VMAX[f], train.POT[f], train.PER[f], train.SHDC_FT[f]])
        train_ships.append(ships)
    except Exception as e:
        pass

for f in range(len(test.GIS_ID)) :
    filename = "IMERG_CSV/" + test.GIS_ID[f] + ".csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[40:81]
        temp = temp.iloc[:, 40:81]
        temp = np.array(temp)
        test_img.append(temp)
        lab = test.IC[f] + test.VMAX[f]
        test_label.append(lab)
        ships = np.array([test.VMAX[f], test.POT[f], test.PER[f], test.SHDC_FT[f]])
        test_ships.append(ships)
    except Exception as e:
        pass

In [185]:
print(len(train_img))
print(len(train_ships))
print(len(train_label))
print(len(test_img))
print(len(test_ships))
print(len(test_label))

9160
9160
9160
430
430
430


In [186]:
test_ships = np.float64(test_ships)
test_label = np.float64(test_label)

In [187]:
X_train_img = train_img
X_train_ships = train_ships
y_train = train_label
X_test_img = test_img
X_test_ships = test_ships
y_test = test_label

In [188]:
X_train_img = np.array(X_train_img)
X_train_img = X_train_img.reshape(-1,41,41,1)
X_train_img = X_train_img.astype('float32')
X_train_ships = np.array(X_train_ships)
X_train_ships = X_train_ships.reshape(-1,4)
y_train = np.array(y_train)

In [189]:
X_test_img = np.array(X_test_img)
X_test_img = X_test_img.reshape(-1,41,41,1)
X_test_img = X_test_img.astype('float32')
X_test_ships = np.array(X_test_ships)
X_test_ships = X_test_ships.reshape(-1,4)
y_test = np.array(y_test)

In [190]:
print(X_train_img.shape)
print(X_train_ships.shape)

(9160, 41, 41, 1)
(9160, 4)


In [136]:
ships_input = keras.Input(shape =(4,), name = "ships_layer")
img_input = keras.Input(shape =(41, 41, 1), name = "img_layer")

w = keras.layers.Conv2D(64,8) (img_input)
w = keras.layers.Conv2D(64,8)(w)
w = keras.layers.Conv2D(64,1)(w)
w = keras.layers.BatchNormalization()(w)
w = keras.activations.relu(w)
w = keras.layers.MaxPool2D(2,2)(w)
w = keras.layers.Conv2D(64, 3)(w)
w = keras.layers.Conv2D(64, 3)(w)
w = keras.layers.Conv2D(256,1)(w)
w = keras.layers.BatchNormalization()(w)
img_output1 = keras.layers.Flatten()(w)

merged_model1 = keras.layers.concatenate([img_output1, ships_input])
output_layer1 = keras.layers.Dense(256)(merged_model1)
output_layer1 = keras.layers.Dense(165)(output_layer1)

new_model1 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer1, name = "model_1")

new_model1.summary()

x = keras.layers.Conv2D(256, 8) (img_input)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(128,1, activation = 'relu')(x)
x = keras.layers.Conv2D(128,5)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(64,1, activation = 'relu')(x)
x = keras.layers.Conv2D(64,3)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
img_output2 = keras.layers.Flatten()(x)

merged_model2 = keras.layers.concatenate([img_output2, ships_input])
output_layer2 = keras.layers.Dense(165)(merged_model2)

new_model2 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer2, name = "model_2")

new_model2.summary()

y = keras.layers.Conv2D(256, (6,2))(img_input)
y = keras.layers.Conv2D(256, (2,6))(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(128,4)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(64,3)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
img_output3 = keras.layers.Flatten()(y)

merged_model3 = keras.layers.concatenate([img_output3, ships_input])
output_layer3 = keras.layers.Dense(165)(merged_model3)

new_model3 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer3, name = "model_3")

new_model3.summary()

z = keras.layers.Conv2D(256, 7)(img_input)
z = keras.layers.MaxPool2D(2,2)(z)
img_output4 = keras.layers.Flatten()(z)

merged_model4 = keras.layers.concatenate([img_output4, ships_input])
output_layer4 = keras.layers.Dense(165)(merged_model4)

new_model4 = keras.Model(inputs = [img_input, ships_input], outputs = output_layer4, name = "model_4")

new_model4.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 img_layer (InputLayer)      [(None, 41, 41, 1)]          0         []                            
                                                                                                  
 conv2d_48 (Conv2D)          (None, 34, 34, 64)           4160      ['img_layer[0][0]']           
                                                                                                  
 conv2d_49 (Conv2D)          (None, 27, 27, 64)           262208    ['conv2d_48[0][0]']           
                                                                                                  
 conv2d_50 (Conv2D)          (None, 27, 27, 64)           4160      ['conv2d_49[0][0]']           
                                                                                            

In [34]:
# ships_input = keras.Input(shape =(4,), name = "ships_layer")
img_input = keras.Input(shape =(41, 41, 1), name = "img_layer")

w = keras.layers.Conv2D(64,8) (img_input)
w = keras.layers.Conv2D(64,8)(w)
w = keras.layers.Conv2D(64,1)(w)
w = keras.layers.BatchNormalization()(w)
w = keras.activations.relu(w)
w = keras.layers.MaxPool2D(2,2)(w)
w = keras.layers.Conv2D(64, 3)(w)
w = keras.layers.Conv2D(64, 3)(w)
w = keras.layers.Conv2D(256,1)(w)
w = keras.layers.BatchNormalization()(w)
img_output1 = keras.layers.Flatten()(w)

merged_model1 = keras.layers.concatenate([img_output1])
output_layer1 = keras.layers.Dense(256)(merged_model1)
output_layer1 = keras.layers.Dense(165)(output_layer1)

new_model1 = keras.Model(inputs = [img_input], outputs = output_layer1, name = "model_1")

new_model1.summary()

x = keras.layers.Conv2D(256, 8) (img_input)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(128,1, activation = 'relu')(x)
x = keras.layers.Conv2D(128,5)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
x = keras.layers.Conv2D(64,1, activation = 'relu')(x)
x = keras.layers.Conv2D(64,3)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = keras.layers.MaxPool2D(2,2)(x)
img_output2 = keras.layers.Flatten()(x)

merged_model2 = keras.layers.concatenate([img_output2])
output_layer2 = keras.layers.Dense(165)(merged_model2)

new_model2 = keras.Model(inputs = [img_input], outputs = output_layer2, name = "model_2")

new_model2.summary()

y = keras.layers.Conv2D(256, (6,2))(img_input)
y = keras.layers.Conv2D(256, (2,6))(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(128,4)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
y = keras.layers.Conv2D(64,3)(y)
y = keras.layers.BatchNormalization()(y)
y = keras.activations.relu(y)
y = keras.layers.MaxPool2D(2,2)(y)
img_output3 = keras.layers.Flatten()(y)

merged_model3 = keras.layers.concatenate([img_output3])
output_layer3 = keras.layers.Dense(165)(merged_model3)

new_model3 = keras.Model(inputs = [img_input], outputs = output_layer3, name = "model_3")

new_model3.summary()

z = keras.layers.Conv2D(256, 7)(img_input)
z = keras.layers.MaxPool2D(2,2)(z)
img_output4 = keras.layers.Flatten()(z)

merged_model4 = keras.layers.concatenate([img_output4])
output_layer4 = keras.layers.Dense(165)(merged_model4)

new_model4 = keras.Model(inputs = [img_input], outputs = output_layer4, name = "model_4")

new_model4.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img_layer (InputLayer)      [(None, 41, 41, 1)]       0         
                                                                 
 conv2d_16 (Conv2D)          (None, 34, 34, 64)        4160      
                                                                 
 conv2d_17 (Conv2D)          (None, 27, 27, 64)        262208    
                                                                 
 conv2d_18 (Conv2D)          (None, 27, 27, 64)        4160      
                                                                 
 batch_normalization_8 (Bat  (None, 27, 27, 64)        256       
 chNormalization)                                                
                                                                 
 tf.nn.relu_7 (TFOpLambda)   (None, 27, 27, 64)        0         
                                                           

In [137]:
new_model1.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model2.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model3.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])
new_model4.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])

In [138]:
new_model3.fit([X_train_img, X_train_ships], y_train, epochs=3, batch_size=8, validation_split = 0.1)
res = new_model3.evaluate([X_test_img, X_test_ships], y_test)
print("MAE = " + str(res[0]))
print("RMSE = " + str((res[2]) ** 0.5))
preds = new_model3.predict([X_test_img, X_test_ships])
a = np.average(preds, axis = 1)
test['preds'] = a

Epoch 1/3
1031/1031 [==============================] - 34s 32ms/step - loss: 13.8635 - mae: 13.8635 - mse: 466.5575 - val_loss: 11.2190 - val_mae: 11.2190 - val_mse: 195.3065
Epoch 2/3
1031/1031 [==============================] - 27s 27ms/step - loss: 5.6177 - mae: 5.6177 - mse: 58.4036 - val_loss: 7.6259 - val_mae: 7.6259 - val_mse: 93.0278
Epoch 3/3
14/14 [==============================] - 1s 47ms/step - loss: 3.7222 - mae: 3.7222 - mse: 22.8950
MAE = 3.7221944332122803
RMSE = 4.7848700488390845
14/14 [==============================] - 0s 24ms/step


In [10]:
train_no_resample = pd.read_csv("DEV/P06_2018_train.csv")
train_no_resample = train_no_resample[["GIS_ID", "DATE", "SHIPS_PER", "SHIPS_POT", "VMAX", "SHDC", "IC", "Category"]]
train_no_resample.columns = ["GIS_ID", "DATE", "PER", "POT", "VMAX", "SHDC_FT", "IC", "Category"]
train_no_resample

,GIS_ID,DATE,PER,POT,VMAX,SHDC_FT,IC,Category
0,ATL_200002_TD_2000062318,2000062318,9.310000e-09,84.0,30,25.300001,0,TD
1,ATL_200002_TD_2000062400,2000062400,0.000000e+00,84.0,30,18.400000,0,TD
2,ATL_200002_TD_2000062406,2000062406,0.000000e+00,84.0,30,16.100000,0,TD
3,ATL_200002_TD_2000062412,2000062412,0.000000e+00,85.0,30,10.100000,0,TD
4,ATL_200002_TD_2000062418,2000062418,0.000000e+00,87.0,30,4.300000,0,TD
...,...,...,...,...,...,...,...,...
3858,ATL_201719_TD_2017110618,2017110618,0.000000e+00,79.0,30,21.400000,5,TD
3859,ATL_201719_TS_2017110700,2017110700,5.000000e+00,62.0,35,22.200001,5,TS
3860,ATL_201719_TS_2017110706,2017110706,1.000000e+01,51.0,40,19.500000,0,TS
3861,ATL_201719_TS_2017110712,2017110712,5.000000e+00,42.0,40,17.600000,0,TS


In [141]:
train_no_resample['Category'].value_counts()['Maj']

504

In [142]:
train_no_resample_TD = train_no_resample[train_no_resample['Category'] == 'TD']
train_no_resample_TD = train_no_resample_TD.sample(90)
train_no_resample_TS = train_no_resample[train_no_resample['Category'] == 'TS']
train_no_resample_TS = train_no_resample_TS.sample(90)
train_no_resample_Min = train_no_resample[train_no_resample['Category'] == 'Min']
train_no_resample_Min = train_no_resample_Min.sample(90)
train_no_resample_Maj = train_no_resample[train_no_resample['Category'] == 'Maj']
train_no_resample_Maj = train_no_resample_Maj.sample(90)
shap_train = pd.concat([train_no_resample_TD, train_no_resample_TS, train_no_resample_Min, train_no_resample_Maj])
shap_train = shap_train.reset_index(drop = True)
shap_train

,GIS_ID,DATE,PER,POT,VMAX,SHDC_FT,IC,Category
0,ATL_201714_TD_2017091606,2017091606,0.000000e+00,110.0,30,13.600000,5,TD
1,ATL_201402_TD_2014072206,2014072206,0.000000e+00,83.0,30,10.500000,0,TD
2,ATL_201214_TD_2012091106,2012091106,0.000000e+00,108.0,25,5.200000,5,TD
3,ATL_201613_TD_2016092006,2016092006,0.000000e+00,98.0,30,3.300000,5,TD
4,ATL_200312_TD_2003090618,2003090618,-5.000000e+00,98.0,25,28.000000,5,TD
...,...,...,...,...,...,...,...,...
355,ATL_201614_C5_2016100100,2016100100,4.500000e+01,-3.0,145,24.000000,-5,Maj
356,ATL_200409_C5_2004091300,2004091300,5.000000e+00,10.0,140,13.300000,0,Maj
357,ATL_200210_C3_2002092206,2002092206,1.860000e-08,26.0,110,10.400001,0,Maj
358,ATL_200518_C5_2005092212,2005092212,-1.000000e+01,-1.0,140,9.200000,-15,Maj


In [143]:
shap_train_img = []
shap_train_ships = []
shap_train_label = []
for f in range(len(shap_train.GIS_ID)) :
    filename = "IMERG_CSV/" + shap_train.GIS_ID[f] + ".csv"
    try:
        temp = pd.read_csv(filename, header = None)
        if (temp.shape != (121,121)):
            continue
        temp = temp[40:81]
        temp = temp.iloc[:, 40:81]
        temp = np.array(temp)
        shap_train_img.append(temp)
        lab = shap_train.IC[f] + shap_train.VMAX[f]
        shap_train_label.append(lab)
        ships = np.array([shap_train.VMAX[f], shap_train.POT[f], shap_train.PER[f], shap_train.SHDC_FT[f]])
        shap_train_ships.append(ships)
    except Exception as e:
        pass

In [144]:
shap_train_img = np.array(shap_train_img)
shap_train_img = shap_train_img.reshape(-1,41,41,1)
shap_train_img = shap_train_img.astype('float32')
shap_train_ships = np.array(shap_train_ships)
shap_train_ships = shap_train_ships.reshape(-1,4)
shap_train_label = np.array(shap_train_label)

In [145]:
len(shap_train_label)

360

In [146]:
#shap.explainers._deep.deep_tf.op_handlers["AddV2"] = shap.explainers._deep.deep_tf.passthrough
shap.explainers._deep.deep_tf.op_handlers["FusedBatchNormV3"] = shap.explainers._deep.deep_tf.linearity_1d(0)
e = shap.DeepExplainer(new_model3, [shap_train_img, shap_train_ships])
shap_values = e.shap_values([X_test_img, X_test_ships])
# shap.summary_plot(shap_values, X_test_img)
# shap.plots.beeswarm(shap_values)

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


In [36]:
shap_values

[[array([[[[ 9.34589843e-06],
           [ 2.09230329e-05],
           [ 3.84609120e-05],
           ...,
           [ 0.00000000e+00],
           [ 0.00000000e+00],
           [ 0.00000000e+00]],
  
          [[ 3.88390654e-05],
           [ 8.56327388e-05],
           [ 1.22837941e-04],
           ...,
           [ 0.00000000e+00],
           [ 0.00000000e+00],
           [ 0.00000000e+00]],
  
          [[ 6.74172406e-05],
           [ 1.44909485e-04],
           [ 1.64309589e-04],
           ...,
           [ 0.00000000e+00],
           [ 0.00000000e+00],
           [ 0.00000000e+00]],
  
          ...,
  
          [[ 0.00000000e+00],
           [ 0.00000000e+00],
           [ 0.00000000e+00],
           ...,
           [ 0.00000000e+00],
           [ 0.00000000e+00],
           [ 0.00000000e+00]],
  
          [[ 0.00000000e+00],
           [ 0.00000000e+00],
           [ 0.00000000e+00],
           ...,
           [ 0.00000000e+00],
           [ 0.00000000e+00],
           [ 0.0

In [147]:
np.shape(shap_values[0][1])

(430, 4)

In [148]:
shap_values[0][1][0][3]

0.48040273840807624

In [149]:
shap_values[0][1]

array([[ -8.90563117,  -0.16686356,   2.58933871,   0.48040274],
       [ -8.90563117,   0.11697661,   1.50204338,   0.31482823],
       [-12.62924002,  -0.09590352,  -1.75984264,  -0.33307196],
       ...,
       [ -5.18202232,   3.0972985 ,   0.41474804,  -1.9600215 ],
       [ -8.90563117,   3.7359389 ,  -1.75984264,  -1.30492225],
       [ -8.90563117,   4.30361926,  -1.75984264,  -1.33371793]])

In [150]:
shap_a = []
for node in range(165):
    for images in range(360):
        shap_a.append(np.sum(shap_values[node][0][images]))

shap_b = []
for node in range(165):
    for images in range(360):
        for ships in range(4):
            shap_b.append(np.sum(shap_values[node][1][images][ships]))

In [151]:
len(shap_b)

237600

In [152]:
shap_b
np.max(shap_b)

shap_VMAX = [shap_b[i] for i in range(0, len(shap_b), 4)]
shap_POT = [shap_b[i] for i in range(1, len(shap_b), 4)]
shap_PER = [shap_b[i] for i in range(2, len(shap_b), 4)]
shap_SHDC = [shap_b[i] for i in range(3, len(shap_b), 4)]

In [178]:
for shaps in [shap_a, shap_VMAX, shap_POT, shap_PER, shap_SHDC]:
    print(np.mean(shaps))

-0.7653736864801305
-2.4051109693869197
0.11550228854727117
-0.051425637656877334
-0.01125721846216789


In [44]:
np.sum(shap_b)

-118791.83350622655

In [31]:
(np.sum(np.sum(np.abs(shap_values[0][0][0]), axis = 0), axis=0))

0.006374581054842565

In [30]:
shap_a
max(shap_a)

0.05565229094727897

In [45]:
shap.plots.force(e.expected_value[0], shap_values[0], [X_test_img[0], X_test_ships[0]])
shap.plots.beeswarm(shap_values)

TypeError: The shap_values arg looks multi output, try `shap_values[i]` instead.

In [68]:
np.shape(shap_values[0][0][0])

(41, 41, 1)

In [100]:
shap_values_nov21 = shap_values
e_nov21 = e